In [1]:
using Distributed
using MachineLearningPotential
using BenchmarkTools
using StaticArrays

In [2]:
atoms = [[0.0000006584,       -0.0000019175,        0.0000000505],
[-0.0000005810,       -0.0000004871,        0.6678432175],
[0.1845874248,       -0.5681026047,        0.2986701538],
[-0.4832557457,       -0.3511072166,        0.2986684497],
[-0.4832557570,        0.3511046452,        0.2986669456],
[0.1845874064,        0.5681000550,        0.2986677202],
[0.5973371920,       -0.0000012681,        0.2986697030],
[-0.1845860897,       -0.5681038901,       -0.2986676192],
[-0.5973358752,       -0.0000025669,       -0.2986696020],
[-0.1845861081,        0.5680987696,       -0.2986700528],
[0.4832570624,        0.3511033815,       -0.2986683486],
[0.4832570738,       -0.3511084803,       -0.2986668445],
[0.0000018978,       -0.0000033480,       -0.6678431165],
[-0.0000017969,        0.0000009162,        1.3230014650],
[0.1871182835,       -0.5758942175,        0.9797717078],
[-0.4898861924,       -0.3559221410,       0.9797699802],
[-0.4898862039,        0.3559224872,        0.9797684555],
[0.1871182648,        0.5758945856,        0.9797692407],
[0.6055300485,        0.0000001908,        0.9797712507],
[0.7926501864,       -0.5758950093,        0.6055339635],
[0.3656681761,       -1.1254128670,        0.5916673591],
[-0.3027660545,       -0.9318173412,        0.6055326929],
[-0.9573332453,       -0.6955436707,        0.5916639831],
[-0.9797705418,       -0.0000006364,        0.6055294407],
[-0.9573332679,        0.6955423392,        0.5916610035],
[-0.3027660847,        0.9318160902,        0.6055287012],
[0.3656681396,        1.1254115783,        0.5916625380],
[0.7926501677,        0.5758937939,        0.6055314964],
[1.1833279992,       -0.0000006311,        0.5916664660],
[0.6770051458,       -0.9318186223,        0.0000033028],
[0.0000006771,       -1.1517907207,        0.0000025175],
[-0.6770037988,       -0.9318186442,        0.0000007900],
[-1.0954155825,       -0.3559242494,       -0.0000012200],
[-1.0954155940,        0.3559203788,       -0.0000027447],
[-0.6770038290,        0.9318147872,       -0.0000032017],
[0.0000006397,        1.1517868856,       -0.0000024165],
[0.6770051155,        0.9318148091,       -0.0000006889],
[1.0954168993,        0.3559204143,        0.0000013211],
[1.0954169108,       -0.3559242139,        0.0000028458],
[0.3027674014,       -0.9318199253,       -0.6055286002],
[-0.3656668229,       -1.1254154134,       -0.5916624370],
[-0.7926488510,       -0.5758976290,       -0.6055313954],
[-1.1833266824,       -0.0000032040,       -0.5916663649],
[-0.7926488697,        0.5758911742,       -0.6055338624],
[-0.3656668594,        1.1254090319,       -0.5916672580],
[0.3027673712,        0.9318135061,       -0.6055325919],
[0.9573345621,        0.6955398357,       -0.5916638820],
[0.9797718586,       -0.0000031986,       -0.6055293396],
[0.9573345846,       -0.6955461743,       -0.5916609025],
[-0.1871169480,       -0.5758984207,       -0.9797691397],
[-0.6055287318,       -0.0000040259,       -0.9797711497],
[-0.1871169667,        0.5758903824,       -0.9797716067],
[0.4898875091,        0.3559183059,       -0.9797698792],
[0.4898875207,       -0.3559263223,       -0.9797683545],
[0.0000031136,       -0.0000047513,       -1.3230013639]]*18.8973*0.36258

positions = [SVector{3}(p[i] for i in 1:3) for p in atoms]
dis2mat = get_distance2_mat(positions)
X = [ 1    1              0.001   0.000  11.338
 1    0              0.001   0.000  11.338
 1    1              0.020   0.000  11.338
 1    0              0.020   0.000  11.338
 1    1              0.035   0.000  11.338
 1    0              0.035   0.000  11.338
 1    1              0.100   0.000  11.338
 1    0              0.100   0.000  11.338
 1    1              0.400   0.000  11.338
 1    0              0.400   0.000  11.338]

radsymmvec = []

for row in eachrow(X)
    symmfunc = RadialType2{Float64}(row[3],row[5],[row[1],row[2]])
    push!(radsymmvec,symmfunc)
end

V = [[0.0001,1,1,11.338],[0.0001,-1,2,11.338],[0.003,-1,1,11.338],[0.003,-1,2,11.338],[0.008,-1,1,11.338],[0.008,-1,2,11.228],[0.008,1,2,11.338],[0.015,1,1,11.338],[0.015,-1,2,11.338],[0.015,-1,4,11.338],[0.015,-1,16,11.338],[0.025,-1,1,11.338],[0.025,1,1,11.338],[0.025,1,2,11.338],[0.025,-1,4,11.338],[0.025,-1,16,11.338],[0.025,1,16,11.338],[0.045,1,1,11.338],[0.045,-1,2,11.338],[0.045,-1,4,11.338],[0.045,1,4,11.338],[0.045,1,16,11.338],[0.08,1,1,11.338],[0.08,-1,2,11.338],[0.08,-1,4,11.338],[0.08,1,4,11.338]]

T = [[1.,1.,1.],[1.,1.,0.],[1.,0.,0.]]

angularsymmvec = []

for element in V 
    for types in T
        symmfunc = AngularType3{Float64}(element[1],element[2],element[3],11.338,types)
        push!(angularsymmvec,symmfunc)
    end
end

total_symm_vec = vcat(radsymmvec,angularsymmvec)
f_mat = cutoff_function.(sqrt.(dis2mat),Ref(total_symm_vec[1].r_cut))

55×55 Matrix{Float64}:
 1.0        0.649134    0.649134    …  0.21115     0.21115     0.0959383
 0.649134   1.0         0.617699       0.0         0.0         0.0
 0.649134   0.617699    1.0            0.00208061  0.0912928   0.0
 0.649134   0.617699    0.617699       0.0         0.00208061  0.0
 0.649134   0.617699    0.223352       0.00208061  0.0         0.0
 0.649134   0.617699    0.223352    …  0.0912928   0.00208061  0.0
 0.649134   0.617699    0.617699       0.0912928   0.0912928   0.0
 0.649134   0.223352    0.617699       0.0912928   0.355548    0.185434
 0.649134   0.223352    0.223352       0.0912928   0.0912928   0.185434
 0.649134   0.223352    0.0889637      0.355548    0.0912928   0.185434
 ⋮                                  ⋱                          
 0.0959383  0.0         0.0            0.623382    0.164805    0.0613471
 0.21115    0.00208061  0.0912928      0.608792    0.608792    0.164805
 0.0959383  0.0         0.185434       0.164805    0.623382    0.0613471
 0.2

In [27]:
calc_one_symm_val(r2_ij,fc_ij,symmfunc::RadialType2) =  ifelse(symmfunc.type_vec==[1.,1.],fc_ij*exp(-symmfunc.eta*r2_ij),0.)

function calc_symm_vals(r2_ij,fc_ij,symm_vector)
    g_vec = calc_one_symm_val.(Ref(r2_ij),Ref(fc_ij),symm_vector)
    return g_vec
end

function calc_symm_val(θ,r_sum,f_ij,f_ik,f_jk,η,λ,ζ,tpz)
    g= (1+λ*θ)^ζ * exp(-η*(r_sum)) * f_ij * f_ik * f_jk    
    return g*tpz
end

function calc_symm_val(θ,r_sum,f_ij,f_ik,f_jk, symmetryfunc)  
    if symmetryfunc.type_vec == [1.,1.,1.]  
        
        return calc_symm_val(θ,r_sum,f_ij,f_ik,f_jk,symmetryfunc.eta,symmetryfunc.lambda,symmetryfunc.zeta,symmetryfunc.tpz)
    else
        return 0.0
    end
end

function calc_symm_vals!(θ,r_ij,r_ik,r_jk,f_ij,f_ik,f_jk,g_vec,symmetry_vector)
    r_sum = r_ij+r_ik+r_jk
    
    g_vec = [calc_symm_val(θ,r_sum,f_ij,f_ik,f_jk,symmfunc) for symmfunc in symmetry_vector]
    
    return g_vec
end


calc_symm_vals! (generic function with 1 method)

In [17]:
function calc_radial_values!(dist2_mat,fc_mat,g_mat,symmetry_vector)
    G,N = size(g_mat)
    last_index = length(symmetry_vector)
    for index1 in 1:N
        for index2 in index1+1:N
            g_vec = calc_symm_vals!(dist2_mat[index1,index2],fc_mat[index1,index2],,symmetry_vector)
            g_mat[1:last_index,i] += g_vec
            g_mat[1:last_index,j] += g_vec
        end
    end
    
    return g_mat
end

  

calc_radial_values! (generic function with 1 method)

In [6]:
function maxspeedtest()
    g = zeros(55)
    for i in 1:55
        for j in i+1:55
            for k in j+1:55
                g[i] += rand()
                g[j] += rand()
                g[k] += rand()
            end
        end
    end

end

function maxspeedtest(index)
    g = zeros(55)
    for j = 1:55
        if j != index 
            for k = j+1:55
                if k != index
                    g[index] += rand()
                    g[j] += rand()
                    g[k] += rand()

                end
            end
        end
    end
end

@benchmark maxspeedtest()

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  76.747 μs … 113.568 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     83.048 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   83.143 μs ±   2.844 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

           ▆    ▇    ▄█    ▂                                    
  ▂▁▁▁▂▁▂▁▃█▃▄▃▃█▅▄▄▄██▄▆▄▄█▆▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂ ▃
  76.7 μs         Histogram: frequency by time         95.7 μs <

 Memory estimate: 496 bytes, allocs estimate: 1.

In [7]:
@benchmark maxspeedtest($rand(1:55))

BenchmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):  4.431 μs …   8.587 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.798 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.850 μs ± 295.557 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

         ▄▇▂▄█▆▃▄▂▁                                            
  ▁▃▂▂▇█▅███████████▇▆▅▅▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  4.43 μs         Histogram: frequency by time        6.17 μs <

 Memory estimate: 496 bytes, allocs estimate: 1.

In [29]:
θ= angular_measure(positions[1],positions[2],positions[3],dis2mat[1,2],dis2mat[1,3])
@benchmark calc_symm_vals!($θ,$dis2mat[1,2],$dis2mat[1,3],$dis2mat[2,3],$f_mat[1,2],$f_mat[1,3],$f_mat[2,3],$zeros(78),$angularsymmvec)

BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  2.775 μs … 346.291 μs  ┊ GC (min … max):  0.00% … 98.73%
 Time  (median):     3.021 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   3.928 μs ±  15.726 μs  ┊ GC (mean ± σ):  19.91% ±  4.92%

   ▅▇██▇▇▅▅▄▃▂▃▃▂▂▂▁              ▁ ▁                         ▃
  ▇███████████████████▇▅▆▆▆▅▁▅▄▆▇███████▇▇▆▆▆▆▆▅▆▆▅▁▆▃▁▃▄▅▆▇▇ █
  2.77 μs      Histogram: log(frequency) by time      5.79 μs <

 Memory estimate: 12.48 KiB, allocs estimate: 238.

# The single calculation

The other notebook has an incredibly efficient implementation of the total symmetry calculation, trying to recreate that for the case where we have picked an atomic index is the next goal